In [1]:
import re
import time
import pickle

In [2]:
class Category:
    def __init__(self, hungarianUrls, slovakUrls, englichCategory, hungarianCategory, slovakCategory):
        self.hungarianUrls = hungarianUrls
        self.slovakUrls = slovakUrls
        self.englishCategory = englishCategory
        self.hungarianCategory = hungarianCategory
        self.slovakCategory = slovakCategory

    def toString(self):
        print("EnglishCategory: " + (self.englishCategory.toString() if hasattr(self.englishCategory, 'label') else "Missing")
              + "\nHungarianCategory: " + (self.hungarianCategory.toString() if hasattr(self.hungarianCategory, 'label') else "Missing")
              + "\nSlovakCategory: " + (self.slovakCategory.toString() if hasattr(self.slovakCategory, 'label') else "Missing"))
        
class Category_Language:
    def __init__(self, dbpediaUri, label, wikipediaUri, broaderCategories):
        self.dbpediaUri = dbpediaUri
        self.label = label
        self.wikipediaUri = wikipediaUri
        self.broaderCategories = broaderCategories

    def toString(self):
        return "\n\tDbPediaUri: " + self.dbpediaUri + "\n\tLabel: " + self.label + "\n\tWikipediaUri: " + self.wikipediaUri + "\n\tBroaderCategories: " + str(self.broaderCategories)
    
def replaceUnicode(string):
    unicode_chars = ["\\u00C1", "\\u00C9", "\\u00CD", "\\u00D3", "\\u00D6", "\\u0150", "\\u00DA", "\\u00DC", "\\u0170", "\\u00E1", "\\u00E9", "\\u00ED",
                    "\\u00F3", "\\u00F6", "\\u0151", "\\u00FA", "\\u00FC", "\\u0171", "\\u00C4", "\\u00D4", "\\u00DD", "\\u010C", "\\u010E", "\\u0139",
                    "\\u013D", "\\u0147", "\\u0154", "\\u0160", "\\u0164", "\\u017D", "\\u01F1", "\\u01C4", "\\u00E4", "\\u00F4", "\\u00FD", "\\u010D",
                    "\\u010F", "\\u013A", "\\u013E", "\\u0148", "\\u0155", "\\u0161", "\\u0165", "\\u017E", "\\u01F3", "\\u01C6"]
    ascii_chars = ["Á", "É", "Í", "Ó", "Ö", "Ő", "Ú", "Ü", "Ű", "á", "é", "í", "ó", "ö", "ő", "ú", "ü", "ű", "Ä",
                   "Ô", "Ý", "Č", "Ď", "Ĺ", "Ľ", "Ň", "Ŕ", "Š", "Ť", "Ž", "DZ", "DŽ", "ä", "ô", "ý", "č", "ď",
                  "ĺ", "ľ", "ň", "ŕ", "š", "ť", "ž", "dz", "dž"]
    for i in range(0, len(unicode_chars)):
        string = string.replace(unicode_chars[i], ascii_chars[i])
        
    return string

In [3]:
category_en = open("Data/category_labels_en.nq", "r")
category_uris_hu = open("Data/category_labels_en_uris_hu.nq", "r")
category_uris_sk = open("Data/category_labels_en_uris_sk.nq", "r")
category_hu = open("Data/category_labels_hu.nq", "r")
category_sk = open("Data/category_labels_sk.nq", "r")
skos_category_en = open("Data/skos_categories_en.nq", "r")
skos_category_hu = open("Data/skos_categories_hu.nq", "r")
skos_category_sk = open("Data/skos_categories_sk.nq", "r")

In [4]:
def file_len(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    return i + 1

In [5]:
category_labels_en_len = file_len("Data/category_labels_en.nq")
category_labels_hu_len = file_len("Data/category_labels_hu.nq")
category_labels_en_uris_hu_len = file_len("Data/category_labels_en_uris_hu.nq")
category_labels_en_uris_sk_len = file_len("Data/category_labels_en_uris_sk.nq")
category_labels_sk_len = file_len("Data/category_labels_sk.nq")
skos_categories_en_len = file_len("Data/skos_categories_en.nq")
skos_categories_hu_len = file_len("Data/skos_categories_hu.nq")
skos_categories_sk_len = file_len("Data/skos_categories_sk.nq")

In [6]:
categoriesDict = {}

In [7]:
# category_en
start_time = time.time()
for i in range(0,category_labels_en_len):
    line = category_en.readline()
    if (line.startswith('#') == False):
        urls = re.compile("""\<(.*?)\>""")
        label = re.compile("""\"(.*?)\"""")                 
        split_urls = urls.findall(line)
        split_label = label.findall(line)
        englishCategory = Category_Language(split_urls[0], split_label[0], split_urls[2], [])
        category = Category([], [], englishCategory, {}, {})
        categoriesDict[category.englishCategory.label] = category
print("--- %s seconds ---" % (time.time() - start_time))

--- 24.36811137199402 seconds ---


In [8]:
uriDict = {}

for key in categoriesDict:
    uriDict[categoriesDict[key].englishCategory.dbpediaUri] = key

In [9]:
# find url to hungarian version of every category
start_time = time.time()
urls = re.compile("""\<(.*?)\>""")
for i in range(0,category_labels_en_uris_hu_len):
    line = category_uris_hu.readline()
    if (line.startswith('#') == False):
        split_urls = urls.findall(line)
        dbPediaUri = split_urls[0]
        if(dbPediaUri in uriDict):
            categoriesDict[uriDict[dbPediaUri]].hungarianUrls.append(split_urls[2])
            
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.23651504516601562 seconds ---


In [10]:
wikiUriDict = {}

for key in categoriesDict:
    if(len(categoriesDict[key].hungarianUrls) > 0):
        wikiUriDict[categoriesDict[key].hungarianUrls[0]] = key

In [11]:
# find hungarian version of every category by wiki url
start_time = time.time()
urls = re.compile("""\<(.*?)\>""")
label = re.compile("""\"(.*?)\"""") 
for i in range(0,category_labels_hu_len):
    line = category_hu.readline()
    if (line.startswith('#') == False):
        split_urls = urls.findall(line)
        split_label = label.findall(line)
        wikiPediaUri = split_urls[2]
        if(wikiPediaUri in wikiUriDict):
            hungarianCategory = Category_Language(split_urls[0], split_label[0], wikiPediaUri, [])
            categoriesDict[wikiUriDict[wikiPediaUri]].hungarianCategory = hungarianCategory
            
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.6105246543884277 seconds ---


In [12]:
# find url to slovak version of every category
start_time = time.time()
urls = re.compile("""\<(.*?)\>""")
for i in range(0,category_labels_en_uris_sk_len):
    line = category_uris_sk.readline()
    if (line.startswith('#') == False):
        split_urls = urls.findall(line)
        dbPediaUri = split_urls[0]
        if(dbPediaUri in uriDict):
            categoriesDict[uriDict[dbPediaUri]].slovakUrls.append(split_urls[2])
            
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.3709993362426758 seconds ---


In [13]:
wikiUriDictSlovak = {}

for key in categoriesDict:
    if(len(categoriesDict[key].slovakUrls) > 0):
        wikiUriDictSlovak[categoriesDict[key].slovakUrls[0]] = key

In [14]:
# find slovak version of every category by wiki url
start_time = time.time()
urls = re.compile("""\<(.*?)\>""")
label = re.compile("""\"(.*?)\"""") 
for i in range(0,category_labels_sk_len):
    line = category_sk.readline()
    if (line.startswith('#') == False):
        split_urls = urls.findall(line)
        split_label = label.findall(line)
        wikiPediaUri = split_urls[2]
        if(wikiPediaUri in wikiUriDictSlovak):
            slovakCategory = Category_Language(split_urls[0], split_label[0], wikiPediaUri, [])
            categoriesDict[wikiUriDictSlovak[wikiPediaUri]].slovakCategory = slovakCategory
            
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.6140003204345703 seconds ---


In [15]:
# find english broader categories
start_time = time.time()
urls = re.compile("""\<(.*?)\>""")
label = re.compile("""\"(.*?)\"""")
for i in range(0, skos_categories_en_len):
    line = skos_category_en.readline()
    if (line.startswith('#') == False):
        if(re.search("#prefLabel", line)):
            split_label = label.findall(line)
        if(re.search("#broader", line) or re.search('#related', line)):
            exception = split_label[0].replace(")", "\)").replace("(", "\(").replace("*", "\*").replace("+", "\+").replace(" ", "_")
            regex = """(?!resource\/Category:""" + exception + """>)resource\/Category:(.*?)\>"""
            broader = re.compile(regex)
            broader_category = broader.findall(line)
            if(len(broader_category) > 0):
                categoriesDict[split_label[0]].englishCategory.broaderCategories.append(broader_category[0].replace("_", " "))

print("--- %s seconds ---" % (time.time() - start_time))

--- 285.02931094169617 seconds ---


In [16]:
labelDictHungarian = {}

for key in categoriesDict:
    if(hasattr(categoriesDict[key], 'hungarianCategory') & hasattr(categoriesDict[key].hungarianCategory, 'label')):
        labelDictHungarian[categoriesDict[key].hungarianCategory.label] = key

In [17]:
# find hungarian broader categories
start_time = time.time()
urls = re.compile("""\<(.*?)\>""")
label = re.compile("""\"(.*?)\"""")
for i in range(0, skos_categories_hu_len):
    line = skos_category_hu.readline()
    if (line.startswith('#') == False):
        if(re.search("#prefLabel", line)):
            split_label = label.findall(line)
        if((re.search("#broader", line) or re.search('#related', line)) and split_label[0] in labelDictHungarian):
            exception = replaceUnicode(split_label[0]).replace(")", "\)").replace("(", "\(").replace("*", "\*").replace("+", "\+").replace(" ", "_")
            line = replaceUnicode(line)
            regex = "(?!resource\/Kategória:" + exception + ">)resource\/Kategória:(.*?)\>"
            broader = re.compile(regex)
            broader_category = broader.findall(line)
            if(len(broader_category) > 0):
                categoriesDict[labelDictHungarian[split_label[0]]].hungarianCategory.broaderCategories.append(replaceUnicode(broader_category[0]).replace("_", " "))
                categoriesDict[labelDictHungarian[split_label[0]]].hungarianCategory.label = replaceUnicode(split_label[0])
            
print("--- %s seconds ---" % (time.time() - start_time))

--- 10.654859066009521 seconds ---


In [18]:
labelDictSlovak = {}

for key in categoriesDict:
    if(hasattr(categoriesDict[key], 'slovakCategory') & hasattr(categoriesDict[key].slovakCategory, 'label')):
        labelDictSlovak[categoriesDict[key].slovakCategory.label] = key

In [19]:
# find slovak broader categories
start_time = time.time()
urls = re.compile("""\<(.*?)\>""")
label = re.compile("""\"(.*?)\"""")
for i in range(0, skos_categories_sk_len):
    line = skos_category_sk.readline()
    if (line.startswith('#') == False):
        if(re.search("#prefLabel", line)):
            split_label = label.findall(line)
        if((re.search("#broader", line) or re.search('#related', line)) and split_label[0] in labelDictSlovak):
            exception = replaceUnicode(split_label[0]).replace(")", "\)").replace("(", "\(").replace("*", "\*").replace("+", "\+").replace(" ", "_")
            line = replaceUnicode(line)
            regex = """(?!resource\/Kategória:""" + exception + """>)resource\/Kategória:(.*?)\>"""
            broader = re.compile(regex)
            broader_category = broader.findall(line)
            if(len(broader_category) > 0):
                categoriesDict[labelDictSlovak[split_label[0]]].slovakCategory.broaderCategories.append(replaceUnicode(broader_category[0]).replace("_", " "))
                categoriesDict[labelDictSlovak[split_label[0]]].slovakCategory.label = replaceUnicode(split_label[0])

            
print("--- %s seconds ---" % (time.time() - start_time))

--- 11.818929433822632 seconds ---


In [20]:
for key in categoriesDict:
    print(categoriesDict[key].toString())
    break

EnglishCategory: 
	DbPediaUri: http://dbpedia.org/resource/Category:Futurama
	Label: Futurama
	WikipediaUri: http://en.wikipedia.org/wiki/Category:Futurama?oldid=547808601
	BroaderCategories: ['Television series created by Matt Groening', 'Comic science fiction', 'Wikipedia categories named after American animated television series']
HungarianCategory: 
	DbPediaUri: http://hu.dbpedia.org/resource/Kateg\u00F3ria:Futurama
	Label: Futurama
	WikipediaUri: http://hu.wikipedia.org/wiki/Kateg\u00F3ria:Futurama?oldid=13435155
	BroaderCategories: ['Rajzfilmek']
SlovakCategory: Missing
None


In [21]:
categoriesDict["Mathematics"].toString()

EnglishCategory: 
	DbPediaUri: http://dbpedia.org/resource/Category:Mathematics
	Label: Mathematics
	WikipediaUri: http://en.wikipedia.org/wiki/Category:Mathematics?oldid=543792969
	BroaderCategories: ['Main topic classifications', 'Abstraction', 'Formal sciences', 'Structure']
HungarianCategory: 
	DbPediaUri: http://hu.dbpedia.org/resource/Kateg\u00F3ria:Matematika
	Label: Matematika
	WikipediaUri: http://hu.wikipedia.org/wiki/Kateg\u00F3ria:Matematika?oldid=13104890
	BroaderCategories: ['Tudomány']
SlovakCategory: 
	DbPediaUri: http://sk.dbpedia.org/resource/Kateg\u00F3ria:Matematika
	Label: Matematika
	WikipediaUri: http://sk.wikipedia.org/wiki/Kateg\u00F3ria:Matematika?oldid=5462139
	BroaderCategories: ['Prírodné vedy']
